# Database parsing

In [279]:
import pandas, numpy as np
import parsingDatabaseUtils, re
import xml, itertools, collections, xml.etree.ElementTree as ET
import tqdm, importlib

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)

procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)
registrosByCaso = registros.groupby('Caso')

C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,27,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\CMRC\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
parsingDatabaseUtils.cleanString('as\n ff,f d ')

'as\n ff f d'

In [12]:
classificationProcedures = {'H0165': 'o', 'H0193': 'o', 'H2120': 'o', 'H2379': 'o', 'H2383': 'o', 'H2386': 'o', 'H2407': 'o', 'H2415': 'o', 'H2595': 'o', 'H2684': 'o', 'H2849': 'o', 'H2852': 'o', 'H2880': 'o', 'H2882': 'o', 'H2884': 'o', 'H2892': 'o', 'H2901': 'o', 'H2904': 'o', 'H2910': 'o', 'H2916': 'o', 'H2959': 'o', 'H2963': 'o', 'H2974': 'a', 'H2975': 'a', 'H3038': 'o', 'H3065': 'o', 'H3066': 'o', 'H3078': 'o', 'H3089': 'p', 'H3092': 'p', 'H3094': 'p', 'H3099': 'a', 'H3100': 'a', 'H3108': 'o', 'H3109': 'o', 'H3111': 'o', 'H3114': 'o', 'H3118': 'o', 'H4421': 'o', 'H4494': 'o', 'H4496': 'o', 'HE020': 'o'}

## Select a case
Identify all  cases with an associated procedure

In [13]:
# Identify all  cases with an associated procedure

In [240]:
interventionToCase = {}
caseToProcedureBirths = {}
for i,r  in tqdm.tqdm_notebook(registros.iterrows()):
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcionProcedimiento = et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo')
        interventionToCase[idDescripcionProcedimiento] = (r.Caso, r.NumeroHistoria, i)
        procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion)[0]
        if classificationProcedures[procedureType] == 'p':
            if r.Caso in caseToProcedureBirths:
                print('Error: two procedures in same case', r.Caso, procedureType)
            caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]

Error: two procedures in same case AD342920 H3094
Error: two procedures in same case AD346540 H3089
Error: two procedures in same case AD350190 H3094
Error: two procedures in same case AD351571 H3094
Error: two procedures in same case AD350755 H3094
Error: two procedures in same case AD353436 H3094
Error: two procedures in same case AD353902 H3089
Error: two procedures in same case AD354284 H3089
Error: two procedures in same case AD356781 H3089
Error: two procedures in same case AD359075 H3094
Error: two procedures in same case NN H3089
Error: two procedures in same case NN H3089
Error: two procedures in same case AD365893 H3094
Error: two procedures in same case AD366060 H3089
Error: two procedures in same case AD363741 H3089
Error: two procedures in same case AD363741 H3089



In [256]:
c

'AD284225'

In [241]:
importlib.reload(parsingDatabaseUtils)

<module 'parsingDatabaseUtils' from 'C:\\Users\\CMRC\\toolsCMRC\\parsingDatabaseUtils.py'>

In [285]:
class BirthDataset:
    """
    Data structure with the information regarding a case, including:
    
    - Procedure
    - Epicrisis
    - Registro de urgencias
    - newborn registration / updates
    - newborn discharge
    """
    posibilitiesAlta = ['alta', 'salida', 'egreso', 'remision', 'traslado']
    def __init__(self, caso, casoDesc, procedure, registros, pacientes, ordenNacimiento = 0):
        self.casoID = caso 
        self.casoDesc = casoDesc
        self.procedure = procedure
        self.procTypeId = procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedure.XmlDescripcion)[0]
        #Split between mother and newborn
        self.registersMother = []
        self.registersNewborn = []
        self.registersUnassigned = []
        self.epicrisis = None
        self.ingreso = None
        self.motherData = pacientes.loc[str(registros.iloc[0].NumeroHistoria)]
        self.registrosRecienNacido = collections.defaultdict(list)
        #Assign each register to mother/newborn or unknown
        for rId, reg in registros.iterrows():
            patient, newbornRegisterRoot = parsingDatabaseUtils.isMaternalRegister(reg, registros, rId)
            if patient == 'mother':
                self.registersMother.append(reg)
            elif patient == 'newborn':
                self.registersNewborn.append(reg)
                self.registrosRecienNacido[newbornRegisterRoot].append(reg)
            #Ignore the ones that are registro de incapacidad (#46), or a link to the description of a procedure(#145)
            elif reg.CodigoRegistro not in [145, 46]:
                #print(caso, reg.Asunto)
                self.registersUnassigned.append(reg)
            if 'Epicrisis' in str(reg.Asunto):
                self.epicrisis = reg
            if 'Ingreso de Urgencias' == reg.Asunto:
                self.ingreso = reg
                

        #Find discharge of baby
        self.newbornDischarge = list(filter(lambda s:
                                       any([p in parsingDatabaseUtils.remove_diacritics(str(s.Asunto)).lower()
                                            for p in BirthDataset.posibilitiesAlta]),
                                       self.registersNewborn
                                      )
                                  )
        if self.newbornDischarge:
            self.newbornDischarge = self.newbornDischarge[0]
            
        elif len(self.newbornDischarge) == 0 and len(self.registersNewborn) > 1: 
            i = np.argmax([p.FechaAsignacionRegistro for p in self.registersNewborn])
            self.newbornDischarge = self.registersNewborn[i]
            
    def getNivelN1(self):        
        res = {}
        res['VAR_0006'] = self.motherData.FechaNac
        #Etnia
        if self.motherData.Etnia == 1:
            res['VAR_0011'] = 'B'
        #Raizal, palenquero, negros/mulatos
        elif self.motherData.Etnia in [3,4,5]:   
            res['VAR_0011'] = 'D'
        #Otras etnias?
        elif self.motherData.Etnia in [2]:
            res['VAR_0011'] = 'E'
            
        #Estudios y alfabetiacion
        if self.motherData.Escolaridad in [2, 3, 4,5,6]:
            res['VAR_0012'] = 'B'
        elif self.motherData.Escolaridad in [1, 8]:
            res['VAR_0012'] = 'A'
            
        #TODO: Que hacer con pre-escolar? 
        if self.motherData.Escolaridad in [8, 1]:
            res['VAR_0013'] = 'A'
        elif self.motherData.Escolaridad in [3]:
            res['VAR_0013'] = 'B'
        elif self.motherData.Escolaridad in [4, 5]:
            res['VAR_0013'] = 'C'
        elif self.motherData.Escolaridad in [6]:
            res['VAR_0013'] = 'D'
            
        if self.motherData.EstadoCivil in ['Casado']:
            res['VAR_0015'] = 'A'
        elif self.motherData.EstadoCivil in ['Soltero']:
            res['VAR_0015'] = 'C'
        elif parsingDatabaseUtils.remove_diacritics(
            self.motherData.EstadoCivil) in ['Union Libre']:
            res['VAR_0015'] = 'D'
            
        res['VAR_0018'] = 'CMRC'
        res['VAR_0019'] = self.motherData.Identificacion 
        
        # Antecedentes
        if self.ingreso is not None:
            etIngreso = ET.fromstring(self.ingreso.RegistroXML)
            
            #Antecedentes familiares
            aFamiliares = parsingDatabaseUtils.findInXML("AntecedentesFamiliares", etIngreso)
            antecedentes = parsingDatabaseUtils.parseAntecedentes(aFamiliares)
            if ('None' in antecedentes and len(antecedentes) > 1) or len(antecedentes) == 0:
                """
                Something weird happened
                """
                pass
            else:
                res['VAR_0020'] = 'B' if 'TBC' in antecedentes else 'A'
                res['VAR_0022'] = 'B' if 'Diabetes' in antecedentes else 'A'
                res['VAR_0024'] = 'B' if 'HTA' in antecedentes else 'A'
                res['VAR_0026'] = 'B' if 'Preeclampsia' in antecedentes else 'A'
                res['VAR_0028'] = 'B' if 'Eclampsia' in antecedentes else 'A'
                res['VAR_0030'] = 'B' if 'Otros' in antecedentes else 'A'

            #Personales solo si no hay nada
            #TODO: a bit of parsing could be done, but I do not have time
            
            #Height and weight
            try:
                res['Var_0055'] = float(parsingDatabaseUtils.findInXML("Peso", etIngreso))
                res['Var_0056'] = float(parsingDatabaseUtils.findInXML("Talla", etIngreso)) * 100 - 100
            except:
                pass
        if self.epicrisis is not None:    
            et = ET.fromstring(self.epicrisis.RegistroXML)
            antececedentesText = parsingDatabaseUtils.findInXML('AntecedentesHTML', et)
            antececedentesText = parsingDatabaseUtils.cleanString(antececedentesText).lower()
            antececedentesText = parsingDatabaseUtils.removeWords(antececedentesText, ['a', 'de', 'el', 'que', 'para', 'y'])
            
            # G P C A : Double check, sometimes it is wrong and FUM
            gpca_fum = parsingDatabaseUtils.parseGPCA_and_fum(antececedentesText)
            if gpca_fum['GPCA_OK']:
                gpca_fum['VAR_0040'] = int(res['G'])
                gpca_fum['VAR_0042'] = int(res['P'])
                gpca_fum['VAR_0047'] = int(res['C'])
                gpca_fum['VAR_0041'] = int(res['A'])
                
            if gpca_fum['fum_OK']:
                if  gpca_fum['fum'] in ['?', 'no']:
                    res['VAR_0059'] =  'A'
                    res['VAR_0057'] = '07/06/1954'
                else:
                    res['VAR_0059'] = 'B'
                    res['VAR_0057'] = '/'.join(parsingDatabaseUtils.parseDate(gpca_fum['fum']))
                
            #Echos
            m = parsingDatabaseUtils.parseEchographies(antececedentesText)
            if m is False:
                res['no_echo'] = 'no_echo_confirmed'
                res['VAR_0060'] = 'A'
            elif isinstance(m, list):
                res['no_echo'] =  'echo_confirmed'
                #TODO: parse date
            else:
                res['no_echo'] =  'no_information'
                
            #PARACLINICS
            #TODO
            
            #Fecha ingreso / egreso

            
            
            # MORBILIDAD:
            
        
        #Parto aborto
        res['VAR_0182'] = 'A' if classificationProcedures[self.procTypeId] == 'p'  else ''
        res['VAR_0182'] = 'B' if classificationProcedures[self.procTypeId] == 'a'  else ''

        return res

In [ ]:
def similarNewbornRegister(t1, t2):
    """
    Check whether two registers are actually the same, check dimensions.
    """
    pass

def classifyCase():
    """
    Check whether it is abortion or birth
    """
    pass

In [282]:
def getParto(birthDataset):
    res = {}
    
    etRegistroRecienNacido = ET.fromstring(self.registroRecienNacido)
    
    #Ingreso
    res['VAR_0183'] =birthDataset.casoDesc.FechaHora
    
    #EG
    res['VAR_0183'] = parsingDatabaseUtils.findInXML('EG', etRegistroRecienNacido)

    #Presentacion
    
    #Posicion parto
    
    #Desgarros
    
    #Nacimiento vivo / muerto
    
    #Nacimiento -> hora
    
    # Placenta completa/ retenida
    
    # Peso / medidas / APGAR
    
    #TODO: defectos
    
    # Egreso

def getNewborn():
    pass

def getDischarge():
    # Egreso
    pass

In [249]:
df.index[df['VAR_0057'] != df['VAR_0057']]

Index(['AD284225', 'AD293576', 'AD294448', 'AD301623', 'AD319654', 'AD320174',
       'AD322208', 'AD326251', 'AD331992', 'AD332167',
       ...
       'AD366060', 'AD363741', 'AD367379', 'AD367925', 'AD368626', 'AD368709',
       'AD368734', 'AD368548', 'AD368822', 'AD368769'],
      dtype='object', length=182)

In [300]:
importlib.reload(parsingDatabaseUtils)
# Split in newborn
registrosByCaso = registros.groupby('Caso')
processedDatasets = {}
for c, p in caseToProcedureBirths.items():
    if c not in casos.index:
        print('ERROR!', c)
        continue
    processedDatasets[c] = BirthDataset(c, casos.loc[c], p, registrosByCaso.get_group(c), pacientes)

ERROR! NN


In [321]:
c = 'AD362245'
p = processedDatasets[c]
print(p.registrosRecienNacido.keys())

dict_keys(['1340165'])


In [323]:
pacientes.loc['215569']

Identificacion           VEN24499094
Carnet                           NaN
Nom1Afil                    JENNIFER
Nom2Afil                    CAROLINA
Ape1Afil                     TORRES 
                            ...     
ParentescoResponsable            NaN
DirResponsable                   NaN
Creencia                         NaN
NombreResponsable                NaN
EstadoEmbarazo                   NaN
Name: 215569, Length: 86, dtype: object

In [322]:
registrosByCaso.get_group(c)

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1340149,215569,AD362245,7,Ingreso de Urgencias,2019-11-19 03:44:12.377,NaN,NaN,ARIVERO,77185537,1042019,...,False,NaN,NaN,NaN,3.0,NaN,2019-11-19 03:29:00.000,NaN,False,NaN
1340165,215569,AD362245,165,Registro del recién nacido,2019-11-19 05:01:24.933,NaN,Padre,sa,1143359181,1042019,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1340167,215569,AD362245,145,NaN,2019-11-19 05:13:46.640,NaN,NaN,RTRESPALACIOS,1047403836,1042019,...,False,NaN,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN
1341050,215569,AD362245,106,Evoluci&#243;n,2019-11-20 11:51:51.167,94.0,1340539,JRAMIREZ,9068204,1042019,...,True,NaN,NaN,NaN,1.0,NaN,2019-11-20 11:49:00.000,NaN,False,NaN
1340539,215569,AD362245,94,Notas de Ingreso a Piso,2019-11-19 15:44:00.580,NaN,NaN,HCASTRO,7883096,1042019,...,False,NaN,NaN,NaN,1.0,NaN,2019-11-19 15:41:00.000,NaN,False,NaN
1341059,215569,AD362245,450,Epicrisis de la Historia Ingreso de Urgencias,2019-11-20 12:00:54.240,1.0,AD362245,JRAMIREZ,9068204,1042019,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN
1341120,215569,AD362245,166,ALTA NEONATO,2019-11-20 13:04:59.090,165.0,1340165,HARGOZ7,73086292,1042019,...,False,NaN,NaN,NaN,1.0,NaN,2019-11-20 13:00:00.000,NaN,False,NaN
1341054,215569,AD362245,46,Registro de Incapacidad,2019-11-20 11:58:19.483,NaN,NaN,JRAMIREZ,9068204,1042019,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN


In [320]:
for c, p in processedDatasets.items():
    
    if p.procTypeId == 'H3092':
        print(c, p.procTypeId,  len(p.registrosRecienNacido))


AD362245 H3092 1


In [246]:
importlib.reload(parsingDatabaseUtils)
resSIP = {}
for c, p in processedDatasets.items():
    resSIP[c] = p.getNivelN1()

In [248]:
df.no_echo.value_counts(), len(df)

(echo_confirmed       1980
 no_echo_confirmed      51
 Name: no_echo, dtype: int64,
 2044)

In [247]:
df = pandas.DataFrame.from_dict(resSIP, orient = 'index')
df

,VAR_0006,VAR_0012,VAR_0013,VAR_0015,VAR_0018,VAR_0019,VAR_0020,VAR_0022,VAR_0024,VAR_0026,VAR_0028,VAR_0030,Var_0055,Var_0056,VAR_0059,VAR_0057,no_echo,VAR_0182,VAR_0060,VAR_0011
AD182404,1996-03-28 00:00:00,B,C,D,CMRC,VEN25322789,A,A,B,A,A,A,65.0,60.0,A,07/06/1954,echo_confirmed,,NaN,NaN
AD191656,1988-02-20 00:00:00,B,C,D,CMRC,VEN19549077,A,A,A,A,A,A,83.0,70.0,B,5/01/16,echo_confirmed,,NaN,NaN
AD184378,1997-12-02 00:00:00,NaN,NaN,D,CMRC,VEN30139482,A,A,A,A,A,A,58.0,60.0,A,07/06/1954,echo_confirmed,,NaN,NaN
AD206592,1984-07-21 00:00:00,NaN,NaN,D,CMRC,VEN19628508,A,A,B,A,A,A,60.0,56.0,A,07/06/1954,no_echo_confirmed,,A,NaN
AD209823,1982-12-19 00:00:00,B,C,D,CMRC,VEN18281890,A,A,A,A,A,A,80.0,60.0,A,07/06/1954,echo_confirmed,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AD368707,2002-04-18 00:00:00,B,C,C,CMRC,VEN32461184,A,A,A,A,A,A,61.0,60.0,A,07/06/1954,echo_confirmed,,NaN,NaN
AD368548,1995-03-20 00:00:00,B,C,D,CMRC,VEN25184931,NaN,NaN,NaN,NaN,NaN,NaN,50.0,66.0,NaN,NaN,NaN,,NaN,NaN
AD368729,1997-02-06 00:00:00,NaN,NaN,NaN,CMRC,VEN26056578,A,A,A,A,A,A,62.0,57.0,B,3/04/19,echo_confirmed,,NaN,NaN
AD368822,1995-10-18 00:00:00,B,C,D,CMRC,VEN24242369,A,A,A,A,A,A,70.0,55.0,NaN,NaN,NaN,,NaN,NaN


In [17]:
noEpicrsis = []
noIngreso = []

noDischarge = []
for c, p in processedDatasets.items():
    if len(p.registersNewborn) == 0:
        noDischarge.append(c)
    if p.ingreso is None:
        noIngreso.append(c)
print(len(noIngreso), len(noDischarge), len(processedDatasets))

83 58 2044


In [255]:
registrosByCaso.get_group(c)

,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,AdmPlan,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
Id,,,,,,,,,,,,,,,,,,,,,
1030730,184931,AD284225,94,Notas de Ingreso a Piso,2018-04-12 16:00:26.227,NaN,NaN,NCARMONA,45470969,100,...,False,NaN,NaN,NaN,1.0,NaN,2018-04-12 16:01:00.000,NaN,False,NaN
1030770,184931,AD284225,145,NaN,2018-04-12 17:10:45.320,NaN,NaN,OBORRE,9074037,100,...,False,NaN,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN
1030778,184931,AD284225,165,Registro del recién nacido,2018-04-12 17:30:34.953,NaN,Padre,sa,1143342901,100,...,False,NaN,NaN,NaN,3.0,NaN,NaN,NaN,False,NaN
1031203,184931,AD284225,166,EVOLUCION ALTA NEONATO,2018-04-13 13:42:34.610,165.0,1030778,HARGOZ7,73086292,100,...,False,NaN,NaN,NaN,1.0,NaN,2018-04-13 13:28:00.000,NaN,False,NaN
1030919,184931,AD284225,94,Notas de Ingreso a Piso,2018-04-13 04:53:11.990,NaN,NaN,ADVALLE,73543467,100,...,False,NaN,NaN,NaN,1.0,NaN,2018-04-13 04:53:00.000,NaN,False,NaN
1030987,184931,AD284225,106,Evoluci&#243;n DE ALTA,2018-04-13 08:43:41.980,94.0,1030919,JRAMIREZ,9068204,100,...,True,NaN,NaN,NaN,1.0,NaN,2018-04-13 08:35:00.000,NaN,False,NaN
1030990,184931,AD284225,450,Epicrisis de la Historia Notas de Ingreso a Piso,2018-04-13 08:45:14.823,1.0,AD284225,JRAMIREZ,9068204,100,...,False,NaN,NaN,NaN,1.0,NaN,NaN,NaN,False,NaN


In [269]:
casos.CausaEgreso.value_counts()

Alta         6263
Otros        3617
Voluntad        2
Translado       1
Name: CausaEgreso, dtype: int64

In [259]:
casos.columns

Index(['FechaHora', 'Reingreso', 'Paciente', 'Acompañante',
       'DireccionAcompanante', 'TelefonoAcompanante', 'Particular',
       'Diagnostico', 'Prestador', 'Auxiliar1', 'Auxiliar2', 'ViaIngreso',
       'CausaExterna', 'IdPlanAdministradora', 'UnidadFuncionalInicial',
       'UnidadFuncionalActual', 'UnidadFuncionalFinal', 'FechaSalida',
       'DiagnosticoSalida', 'EstadoSalida', 'EstadoCaso', 'Cama', 'Usuario',
       'Autorizacion', 'RemitidaA', 'CiudadRemitida', 'FechaRemision',
       'CausaMuerte', 'FechaHoraMuerte', 'PrestadorDefuncion', 'CausaEgreso',
       'Triage', 'Glasgow', 'Remitido', 'FechaRemitido', 'InstitucionRemitido',
       'MunicipioRemision', 'Catastrofe', 'FechaCatastrofe',
       'DireccionCatastrofe', 'ZonaCatastrofe', 'MunicipioCatastrofe',
       'TipoEventoCatastrofe', 'EventoCatastrofico', 'TipoTratamiento',
       'IdPlanFinal', 'CodigoIntitucionRemitente', 'ProfesionaRemitente',
       'CargoProfesionalRemitente', 'IdTriage', 'CodigoEntidadaRecibe

In [268]:
pacientes.loc['184931']

Identificacion           VEN18862081
Carnet                           NaN
Nom1Afil                     TATIANA
Nom2Afil                    KATIUSCA
Ape1Afil                     GUANIPA
                            ...     
ParentescoResponsable            NaN
DirResponsable                   NaN
Creencia                         NaN
NombreResponsable                NaN
EstadoEmbarazo                   NaN
Name: 184931, Length: 86, dtype: object

In [266]:
parsingDatabaseUtils.prettyPrintXML(processedDatasets['AD284225'].epicrisis.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Notas de Ingreso a Piso" Caso="AD284225" CentroA="01" CodigoDiagnosticoRelacionado1="Z392" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O829" IdPaciente="184931" Modulo="01" Padre="AD284225" PlanAdm="100" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- DESCRIPCI&#211;N DE LA NOTA:
PACIENTE FEMENINA DE 30 A&#209;OS DE EDAD CON DIAGNOSTICOS DE:- POP DE CESAREA TRANSPERITONEAL(12-04-2018- 3:55PM)- PUERPERIO MEDIATO- RECIEN NACIDO UNICO VIVO(MASCULINO, 3060GR, 50CM)
SUBJETIVO:PACIENTE REFIERE SENTIRSE BIEN, NIEGA FIEBRE, DOLOR, SANGRADO POR GENITALES U OTRA SINTOMATOLOGIA ASOCIADAOBJETIVO:PA: 120/80 MMHG FC: 90 LPM FR: 18 LPM T: 36.5C&#176;PACIENTE CON BUENAS CONDICIONES GENERALES, NORMOCEFALO, CON BUENA IMPLANTACION CAPILAR, PUPILAS ISOCORICAS, NORMO

In [250]:
c = 'AD284225'
et = ET.fromstring(processedDatasets[c].epicrisis.RegistroXML)
m = parseEchographies(parsingDatabaseUtils.findInXML('AntecedentesHTML', et))

In [239]:
c = list(processedDatasets.keys())[884]
for c in processedDatasets.keys():
    if 'SEPSIS' in processedDatasets[c].epicrisis.RegistroXML:
        print(c)
        parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

AD218865
<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD218865" CentroA="01" CodigoDiagnosticoRelacionado1="O839" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="Z340" IdPaciente="164688" Modulo="01" Padre="AD218865" PlanAdm="100" Prestador="1047400244" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  7-&amp;gt; MENARQUIA: 14 A&#209;OS.
VIDA MARITAL: 16 A&#209;OS.
VIDA OBSTETRICA: 17 A&#209;OS.
G2 P0 C0 A0  
CICLOS: 5X30
FUM: 22.06.2016
EG X FUM: 39.2
COMPA&#209;EROS SEXUALES: 1
GRUPO SANGUINEO: A POSITIVO
PLANIFICACI&#211;N FAMILIAR: NIEGA
CONTROLES PRENATALES: 6

*ECOGRAFIA: 6.02.2017: EMBARAZO DE 32 SEMANAS
- FAMILIARES:
  NIEGA
- AL&#201;RGICOS:
  
- QUIR&#218;RGICOS:"/>
	<row NombreCampo="AplicaCuidadoEnfermeria" NombreTabla="CamposBoolean" ValorCampo="false"/>
	<row N

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [90]:
c = list(processedDatasets.keys())[244]
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD336169" CentroA="01" CodigoDiagnosticoRelacionado1="Z348" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O623" IdPaciente="206122" Modulo="01" Padre="AD336169" PlanAdm="100" Prestador="1128057935" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  1-&amp;gt;  ASMA

  2-&amp;gt; NIEGA

  5-&amp;gt;  NIEGA

  7-&amp;gt; 
MENARQUIA: 8 A&#209;OS 
INICIO DE VIDA SEXUAL: 15 A&#209;OS
INICIO DE VIDA OBSTETRICA: 15 A&#209;OS 
G1 P0 C0 A0
PERIODOS MENSTRUALES:  REGULARES
FUM:  NO SABE 
PLANIFICACION FAMILIAR: NIEGA
COMPA&#209;EROS SEXUALES: 1
ITS: NIEGA 
GRUPO SANGUINEO: A+
CONTROLES PRENALES: 6

PACIENTE NO TRAE PARACLINICOS 

ECOGRAFIA 
14/1/19 EMB 25.3 SEMANAS PARA HOY 39.6 SEMANAS 
5/9/18 EMB 6.1 SEMANA S PARA HOY 39.2 SEMANAS * 7-&a